In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# Read the data
df = pd.read_csv('data/birdflu.csv')

# Ensure FIPS codes are properly formatted
df['FIPS Codes'] = df['FIPS Codes'].astype(str).str.zfill(5)

# Calculate quantiles for better color distribution
quantiles = np.quantile(df['Flock Size'], [0, 0.2, 0.4, 0.6, 0.8, 1])

# Create choropleth map
fig = px.choropleth_mapbox(df,
    geojson='https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json',
    locations='FIPS Codes',
    color='Flock Size',
    color_continuous_scale="Reds",
    range_color=[df['Flock Size'].min(), df['Flock Size'].max()],
    mapbox_style='carto-positron',  # Light style basemap that doesn't require token
    zoom=3.5,
    center={"lat": 37.0902, "lon": -95.7129},
    opacity=0.7,
    hover_data=['County Name', 'State', 'Flock Type'],
    title="Bird Flu Cases by Flock Size Across US Counties"
)

# Update layout
fig.update_layout(
    title_x=0.5,
    width=1200,
    height=800,
    margin={"r":0,"t":30,"l":0,"b":0},
    coloraxis_colorbar=dict(
        title="Flock Size",
        tickvals=quantiles,
        ticktext=[f"{val:,.0f}" for val in quantiles],
        lenmode="fraction",
        len=0.75
    )
)

# Update traces
fig.update_traces(
    marker_line_color='rgb(200,200,200)',
    marker_line_width=0.5
)

# Show the map
fig.show()

/var/folders/ch/2jdk3nmx6m59lq348r2ydl3m0000gp/T/ipykernel_14520/1638002995.py:15: DeprecationWarning:

*choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [ ]:
# Create summary statistics
state_summary = df.groupby('State').agg({
    'Flock Size': ['sum', 'mean', 'count'],
    'Outbreaks': 'sum'
}).round(2)

# Flatten column names
state_summary.columns = ['Total Flock Size', 'Average Flock Size', 'Number of Cases', 'Total Outbreaks']

# Sort by total flock size
state_summary = state_summary.sort_values('Total Flock Size', ascending=False)

# Display top 10 states
print("Top 10 States by Total Affected Flock Size:")
print(state_summary.head(10))